In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Read Data Function
def read_data(filename):
    
    # Retrieve Data
    data = pd.read_csv(filename)
    
    # Drop Columns
    clean = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Date'])
    
    # Drop Last Row
    clean = clean[:-1].copy()
    
    # Drop missing data
    clean = clean.dropna()
    
    # Rename Columns
    cols = ['Year', 'Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']
    clean.columns = cols
    
    return clean

In [3]:
# Run Model (Machine Learning)
def run_model(clean):
    
    # Create Lag Column
    clean['nextCO2'] = clean.shift(1)['CO2']
    
    # Set Training and Testing Split
    training = clean.loc[(clean['Year']<2010) & (clean['Year']>1990)]
    testing = clean.loc[clean['Year']>=2010]

    # Set Variables
    X_train = training[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_train = training['nextCO2']
    X_test = testing[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_test = testing['nextCO2']
    X = clean[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y = clean['nextCO2']

    # Linear Regression Model
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()

    # Train Modele
    model.fit(X_train, y_train)

    # Score for 1991-2009
    print(f"Train Score: {model.score(X_train, y_train)}")

    # Score for 2010+
    print(f"Test Score: {model.score(X_test, y_test)}")

    # Predictions vs Actual
    predictions = model.predict(X)
    print(f"Actual: {y[1]}")
    print(f"Predicted: {predictions[1]}")
    print(f"Error: {predictions[1]-y[1]}")

    return pd.DataFrame({"Year": clean["Year"],
                         "Predicted": predictions,
                         "Actual": y,
                         "Error": predictions - y,
                         "Country": country})

In [4]:
final = pd.DataFrame(columns = ["Year", "Predicted", "Actual", "Error", "Country"])

for filename in os.listdir("Cleaning/Combined"):
    if ".csv" in filename:
        
        print()
        
        country = filename.replace('.csv', '')
        
        print(country)
        
        clean = read_data("Cleaning/Combined/" + filename)
        
        try:
            output = run_model(clean)
            final = final.append(output)
        
        except:
            print("Error")


Australia
Train Score: 0.9980161199094038
Test Score: -8.598584679087166
Actual: 278424.660375653
Predicted: 277813.73579278856
Error: -610.9245828644489

Canada
Train Score: 0.9720816715902076
Test Score: -30.064083848802717
Actual: 462117.351149139
Predicted: 461495.3489376646
Error: -622.0022114744061

Japan
Train Score: 0.9579849325039395
Test Score: -7.148436447481279
Actual: 438008.80219971295
Predicted: 435577.76341413485
Error: -2431.038785578101

New Zealand
Train Score: 0.6442652320633644
Test Score: -6.494606586769894
Actual: 162385.109422551
Predicted: 165848.15947876355
Error: 3463.0500562125526

Russia
Train Score: 0.9129729599932044
Test Score: -3.3836190014212084
Actual: 169285.872691812
Predicted: 153784.9038846516
Error: -15500.968807160389

Spain
Train Score: 0.44415638635284505
Test Score: -1.3488100164430588
Actual: 44154.420948084815
Predicted: 44916.22984873901
Error: 761.8089006541923

Switzerland
Train Score: 0.8068565921376333
Test Score: -1.5550530620556984


In [5]:
# Export 
final.to_csv('Resources/ML_CO2.csv')